# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'03-25-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'03-24-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-03-25 05:27:00,33.93911,67.709953,56192,2466,49802.0,3924.0,Afghanistan,144.347276,4.388525
1,NaN,NaN,NaN,Albania,2021-03-25 05:27:00,41.15330,20.168300,122295,2171,87193.0,32931.0,Albania,4249.600389,1.775216
2,NaN,NaN,NaN,Algeria,2021-03-25 05:27:00,28.03390,1.659600,116438,3069,81065.0,32304.0,Algeria,265.530742,2.635737
3,NaN,NaN,NaN,Andorra,2021-03-25 05:27:00,42.50630,1.521800,11638,114,11082.0,442.0,Andorra,15062.447421,0.979550
4,NaN,NaN,NaN,Angola,2021-03-25 05:27:00,-11.20270,17.873900,21836,532,20172.0,1132.0,Angola,66.438940,2.436344


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/15/21,3/16/21,3/17/21,3/18/21,3/19/21,3/20/21,3/21/21,3/22/21,3/23/21,3/24/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,55985,55995,56016,56044,56069,56093,56103,56153,56177,56192
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,118017,118492,118938,119528,120022,120541,121200,121544,121847,122295
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,115410,115540,115688,115842,115970,116066,116157,116255,116349,116438


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/15/21,3/16/21,3/17/21,3/18/21,3/19/21,3/20/21,3/21/21,3/22/21,3/23/21,3/24/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,2459,2460,2460,2462,2462,2462,2463,2464,2466,2466
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,2060,2077,2092,2106,2122,2133,2137,2145,2156,2171
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,3040,3045,3048,3051,3053,3055,3057,3061,3066,3069


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/15/21,3/16/21,3/17/21,3/18/21,3/19/21,3/20/21,3/21/21,3/22/21,3/23/21,3/24/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,49481,49499,49536,49664,49681,49681,49737,49743,49752,49802
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,81061,81810,82554,83264,83954,84625,85147,85846,86497,87193
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,79994,80103,80219,80347,80455,80566,80659,80887,80981,81065


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,3/15/21,3/16/21,3/17/21,3/18/21,3/19/21,3/20/21,3/21/21,3/22/21,3/23/21,3/24/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,6471,6474,6483,6495,6498,6510,6513,6517,6525,6533
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,20210,20227,20263,20287,20317,20329,20347,20361,20354,20395
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,2195,2198,2199,2202,2206,2212,2212,2213,2213,2216


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,3/15/21,3/16/21,3/17/21,3/18/21,3/19/21,3/20/21,3/21/21,3/22/21,3/23/21,3/24/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,95,95,96,98,98,98,98,98,99,99
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,294,294,295,296,296,296,296,296,297,300
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,53,53,54,54,54,54,54,54,54,54


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
652,1001.0,Autauga,Alabama,US,2021-03-25 05:27:00,32.539527,-86.644082,6533,99,NaN,NaN,"Autauga, Alabama, US",11693.425692,1.515383
689,1075.0,Lamar,Alabama,US,2021-03-25 05:27:00,33.779950,-88.096680,1378,33,NaN,NaN,"Lamar, Alabama, US",9981.890619,2.394775
690,1077.0,Lauderdale,Alabama,US,2021-03-25 05:27:00,34.901719,-87.656247,9065,217,NaN,NaN,"Lauderdale, Alabama, US",9775.798294,2.393822


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,30009773,545237,0.0
Brazil,12220011,300685,10750131.0
India,11787534,160692,11231650.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,30009773,545237,0.0,29464536.0,2021-03-25 05:27:00,37.936303,-91.379001
Brazil,12220011,300685,10750131.0,1169195.0,2021-03-25 05:27:00,-12.669522,-48.480493
India,11787534,160692,11231650.0,395192.0,2021-03-25 05:27:00,23.088275,81.806127
Russia,4433364,94624,4051703.0,287037.0,2021-03-25 05:27:00,54.546312,62.120860
France,4374774,93083,290310.0,3991381.0,2021-03-25 05:27:00,6.430808,-34.730285


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,3648572,57931,0.0
Texas,2766259,47689,0.0
Florida,2021656,32850,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,3648572,57931,0.0,3590641.0,2021-03-25 05:27:00,37.843962,-120.728594
Texas,2766259,47689,0.0,2718570.0,2021-03-25 05:27:00,31.660643,-98.653069
Florida,2021656,32850,0.0,1988806.0,2021-03-25 05:27:00,28.940755,-82.700744
New York,1816518,49717,0.0,1766801.0,2021-03-25 05:27:00,42.544151,-75.474183
Illinois,1227613,23408,0.0,1204205.0,2021-03-25 05:27:00,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1215736,22960,0.0
Arizona,Maricopa,522323,9601,0.0
Illinois,Cook,490661,9601,0.0
Florida,Miami-Dade,437452,5751,0.0
Texas,Harris,371494,5677,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1215736,22960,0.0,1192776.0,2021-03-25 05:27:00,34.308284,-118.228241,6037.0
Arizona,Maricopa,522323,9601,0.0,512722.0,2021-03-25 05:27:00,33.348359,-112.491815,4013.0
Illinois,Cook,490661,9601,0.0,481060.0,2021-03-25 05:27:00,41.841448,-87.816588,17031.0
Florida,Miami-Dade,437452,5751,0.0,431701.0,2021-03-25 05:27:00,25.611236,-80.551706,12086.0
Texas,Harris,371494,5677,0.0,365817.0,2021-03-25 05:27:00,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-03-22,56153,121544,116255,11545,21757,1072,2252172,184219,29211,516565,...,4315602,84230,81446,3,151894,2575,225976,3516,86535,36684
2021-03-23,56177,121847,116349,11591,21774,1080,2261577,185020,29221,519980,...,4321019,86007,81678,3,152508,2575,228044,3612,86779,36717
2021-03-24,56192,122295,116438,11638,21836,1080,2269877,186184,29230,523269,...,4326645,87812,81678,3,153315,2576,230076,3703,86993,36749


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-03-22,2464,2145,3061,113,530,28,54671,3360,909,9089,...,126411,811,622,0,1502,35,2458,771,1182,1514
2021-03-23,2466,2156,3066,113,530,28,54823,3384,909,9121,...,126523,827,623,0,1511,35,2478,785,1185,1516
2021-03-24,2466,2171,3069,114,532,28,54946,3398,909,9151,...,126621,843,623,0,1521,35,2501,800,1187,1516


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-03-22,49743,85846,80887,11014,20090,701,2037686,168721,22977,475227,...,12225,68983,79884,1,141260,2233,199965,1546,83319,34315
2021-03-23,49752,86497,80981,11050,20099,715,2043957,169172,22982,477940,...,12383,70372,80045,1,141260,2246,201675,1566,83478,34447
2021-03-24,49802,87193,81065,11082,20172,715,2049035,169485,22988,480597,...,12397,71588,80045,1,142381,2265,203340,1577,83617,34476


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,3/15/21,3/16/21,3/17/21,3/18/21,3/19/21,3/20/21,3/21/21,3/22/21,3/23/21,3/24/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,6471,6474,6483,6495,6498,6510,6513,6517,6525,6533
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1365,1366,1367,1368,1369,1372,1372,1373,1372,1378
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,8962,8973,8993,9000,9007,9019,9024,9034,9034,9065
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,2903,2906,2908,2908,2910,2908,2910,2910,2910,2911
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,15197,15205,15218,15231,15248,15274,15282,15290,15292,15333


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-03-22,511779,61435,0,836737,328707,3644488,450934,299667,92429,49,...,801645,2757552,381788,17839,2767,605967,355986,138102,629787,55730
2021-03-23,511789,61611,0,837244,328946,3646941,452304,300565,92609,49,...,803104,2762270,382171,17933,2767,607234,356536,138429,629787,55799
2021-03-24,512711,61830,0,837849,329177,3648572,453520,302022,92819,49,...,804492,2766259,382733,18028,2876,608704,357499,138818,631301,55887


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-03-22        6517   20361    2213  2529   6387    1194   2072   14165   
2021-03-23        6525   20354    2213  2530   6388    1195   2073   14162   
2021-03-24        6533   20395    2216  2535   6402    1197   2077   14186   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-03-22         3469     1794  ...    2640    606     3053      688   
2021-03-23         3470     1794  ...    2641    606     3055      689   
2021-03-24         3471     1797  ...    2641    606     3055      692   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-03-22           3947  3530  2109          0      889    631  
2021-03-23           3951  3532  2111          0      889    632  
2021-03-24           3953  3547  2113          0      890    632  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-03-22,10436,310,0,16745,5544,57622,6070,7841,1531,0,...,11713,47446,2062,219,25,10127,5183,2612,7240,693
2021-03-23,10450,312,0,16798,5547,57798,6074,7845,1535,0,...,11747,47527,2077,220,25,10137,5186,2613,7240,695
2021-03-24,10487,312,0,16842,5560,57931,6077,7852,1535,0,...,11792,47689,2082,222,25,10143,5200,2619,7263,695


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-03-22          98     296      54   58    130      39     66     303   
2021-03-23          99     297      54   58    130      39     66     304   
2021-03-24          99     300      54   58    131      39     66     304   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-03-22          114       42  ...      29     11       31        7   
2021-03-23          114       42  ...      30     11       31        7   
2021-03-24          115       42  ...      30     11       31        7   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-03-22             36     9    12          0       26      5  
2021-03-23             36     9    12          0       26      5  
2021-03-24             36     9    12          0       26      5  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-03-22,0.000891,0.002838,0.000844,0.002431,0.001104,0.037754,0.002850,0.002754,0.000171,0.004691,...,0.001254,0.033028,0.001315,0.0,0.005102,0.001166,0.010454,0.028672,0.000995,0.000518
2021-03-23,0.000427,0.002493,0.000809,0.003984,0.000781,0.007463,0.004176,0.004348,0.000342,0.006611,...,0.001255,0.021097,0.002849,0.0,0.004042,0.000000,0.009151,0.027304,0.002820,0.000900
2021-03-24,0.000267,0.003677,0.000765,0.004055,0.002847,0.000000,0.003670,0.006291,0.000308,0.006325,...,0.001302,0.020987,0.000000,0.0,0.005292,0.000388,0.008911,0.025194,0.002466,0.000872


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-03-22,0.000406,0.003744,0.001308,0.00000,0.005693,0.0,0.002310,0.003584,0.0,0.001653,...,0.000142,0.023990,0.000000,NaN,0.006028,0.0,0.012773,0.026631,0.002545,0.001323
2021-03-23,0.000812,0.005128,0.001633,0.00000,0.000000,0.0,0.002780,0.007143,0.0,0.003521,...,0.000886,0.019729,0.001608,NaN,0.005992,0.0,0.008137,0.018158,0.002538,0.001321
2021-03-24,0.000000,0.006957,0.000978,0.00885,0.003774,0.0,0.002244,0.004137,0.0,0.003289,...,0.000775,0.019347,0.000000,NaN,0.006618,0.0,0.009282,0.019108,0.001688,0.000000


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-03-22,0.000121,0.008209,0.002827,0.003736,0.000847,0.023358,0.003711,0.002567,0.000261,0.003951,...,0.004024,0.018500,0.000902,0.0,0.003566,0.015924,0.007731,0.007823,0.000733,0.001342
2021-03-23,0.000181,0.007583,0.001162,0.003269,0.000448,0.019971,0.003078,0.002673,0.000218,0.005709,...,0.012924,0.020135,0.002015,0.0,0.000000,0.005822,0.008551,0.012937,0.001908,0.003847
2021-03-24,0.001005,0.008047,0.001037,0.002896,0.003632,0.000000,0.002484,0.001850,0.000261,0.005559,...,0.001131,0.017280,0.000000,0.0,0.007936,0.008459,0.008256,0.007024,0.001665,0.000842


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-03-22,0.000624,0.005713,NaN,0.000579,0.000158,0.000929,0.000675,0.010031,0.001539,0.0,...,0.003787,0.001066,0.000417,0.005581,0.000000,0.001757,0.003394,0.002003,0.000604,0.002681
2021-03-23,0.000020,0.002865,NaN,0.000606,0.000727,0.000673,0.003038,0.002997,0.001947,0.0,...,0.001820,0.001711,0.001003,0.005269,0.000000,0.002091,0.001545,0.002368,0.000000,0.001238
2021-03-24,0.001802,0.003555,NaN,0.000723,0.000702,0.000447,0.002688,0.004848,0.002268,0.0,...,0.001728,0.001444,0.001471,0.005297,0.039393,0.002421,0.002701,0.002810,0.002404,0.001577


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-03-22      0.000614  0.000688  0.000452  0.000000  0.000627  0.000000   
2021-03-23      0.001228 -0.000344  0.000000  0.000395  0.000157  0.000838   
2021-03-24      0.001226  0.002014  0.001356  0.001976  0.002192  0.001674   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-03-22      0.000000  0.000212  0.000866  0.000000  ... -0.000379   
2021-03-23      0.000483 -0.000212  0.000288  0.000000  ...  0.000379   
2021-03-24      0.001930  0.001695  0.000288  0.001672  ...  0.000000   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-03-22      0.001653  0.001640  0.004380   0.003050  0.005698  0.002853   
2021-03-23      0.000000  0.000655  0.001453   0.001013  0.000567  0.000948   
2021-03-24      0.000000  0.000000  0.004354   0.000506  0.004247  0.000947   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-03-22            NaN  0.000000  0.000000  
2021-03-23            NaN  0.000000  0.001585  
2021-03-24            NaN  0.001125  0.000000  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-03-22,0.000000,0.000000,NaN,0.000000,0.000903,0.002035,-0.000494,0.001149,0.001308,NaN,...,0.000342,0.002112,0.000000,0.000000,0.0,0.000988,0.001739,0.002302,0.000000,0.000000
2021-03-23,0.001342,0.006452,NaN,0.003165,0.000541,0.003054,0.000659,0.000510,0.002613,NaN,...,0.002903,0.001707,0.007274,0.004566,0.0,0.000987,0.000579,0.000383,0.000000,0.002886
2021-03-24,0.003541,0.000000,NaN,0.002619,0.002344,0.002301,0.000494,0.000892,0.000000,NaN,...,0.003831,0.003409,0.002407,0.009091,0.0,0.000592,0.002700,0.002296,0.003177,0.000000


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State   Alabama                                                  \
Admin2           Autauga   Baldwin Barbour Bibb    Blount Bullock Butler   
2021-03-22      0.000000  0.000000     0.0  0.0  0.000000     0.0    0.0   
2021-03-23      0.010204  0.003378     0.0  0.0  0.000000     0.0    0.0   
2021-03-24      0.000000  0.010101     0.0  0.0  0.007692     0.0    0.0   

Province_State                             ...   Wyoming                  \
Admin2         Calhoun  Chambers Cherokee  ...      Park Platte Sheridan   
2021-03-22      0.0000  0.000000      0.0  ...  0.000000    0.0      0.0   
2021-03-23      0.0033  0.000000      0.0  ...  0.034483    0.0      0.0   
2021-03-24      0.0000  0.008772      0.0  ...  0.000000    0.0      0.0   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-03-22          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2021-03-23          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2021-03-24          0.0        0.0   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-03-22,0.000596,0.003870,0.000870,0.002944,0.001695,0.022582,0.002711,0.003518,0.000239,0.005319,...,0.001260,0.026504,0.001211,0.000061,0.005536,0.000755,0.009755,0.030820,0.001680,0.000453
2021-03-23,0.000512,0.003182,0.000839,0.003464,0.001238,0.015022,0.003444,0.003933,0.000291,0.005965,...,0.001258,0.023800,0.002030,0.000031,0.004789,0.000378,0.009453,0.029062,0.002250,0.000676
2021-03-24,0.000389,0.003429,0.000802,0.003759,0.002043,0.007511,0.003557,0.005112,0.000299,0.006145,...,0.001280,0.022394,0.001015,0.000015,0.005040,0.000383,0.009182,0.027128,0.002358,0.000774


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-03-22,0.000339,0.003907,0.001008,0.000017,0.004061,0.004759,0.001672,0.004055,1.423425e-29,0.002306,...,0.000336,0.021579,1.464568e-15,NaN,0.006064,1.237455e-62,0.011085,0.020369,0.001873,0.001091
2021-03-23,0.000576,0.004517,0.001321,0.000009,0.002031,0.002380,0.002226,0.005599,7.117124e-30,0.002913,...,0.000611,0.020654,8.038585e-04,NaN,0.006028,6.187277e-63,0.009611,0.019264,0.002206,0.001206
2021-03-24,0.000288,0.005737,0.001150,0.004429,0.002902,0.001190,0.002235,0.004868,3.558562e-30,0.003101,...,0.000693,0.020000,4.019293e-04,NaN,0.006323,3.093638e-63,0.009446,0.019186,0.001947,0.000603


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-03-22,0.000459,0.007710,0.002052,0.002925,0.000689,0.030022,0.003486,0.002310,0.000200,0.004643,...,0.002967,0.017401,0.000872,0.0,0.003561,0.008482,0.008115,0.005367,0.001653,0.000939
2021-03-23,0.000320,0.007647,0.001607,0.003097,0.000568,0.024997,0.003282,0.002491,0.000209,0.005176,...,0.007946,0.018768,0.001444,0.0,0.001780,0.007152,0.008333,0.009152,0.001781,0.002393
2021-03-24,0.000662,0.007847,0.001322,0.002996,0.002100,0.012498,0.002883,0.002171,0.000235,0.005368,...,0.004538,0.018024,0.000722,0.0,0.004858,0.007806,0.008295,0.008088,0.001723,0.001617


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-03-22,0.000789,0.003090,NaN,0.000607,0.000363,0.000727,0.001570,0.005473,0.002471,7.077726e-113,...,0.002142,0.001048,0.000773,0.007513,0.000002,0.002029,0.002353,0.002317,0.000717,0.001516
2021-03-23,0.000404,0.002977,NaN,0.000606,0.000545,0.000700,0.002304,0.004235,0.002209,3.538863e-113,...,0.001981,0.001380,0.000888,0.006391,0.000001,0.002060,0.001949,0.002342,0.000358,0.001377
2021-03-24,0.001103,0.003266,NaN,0.000665,0.000624,0.000574,0.002496,0.004541,0.002238,1.769431e-113,...,0.001855,0.001412,0.001179,0.005844,0.019697,0.002240,0.002325,0.002576,0.001381,0.001477


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-03-22      0.000798  0.000827  0.000768  0.000668  0.000531  0.000056   
2021-03-23      0.001013  0.000242  0.000384  0.000532  0.000344  0.000447   
2021-03-24      0.001119  0.001128  0.000870  0.001254  0.001268  0.001060   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-03-22      0.000189  0.000319  0.000553  0.000195  ... -0.000019   
2021-03-23      0.000336  0.000053  0.000421  0.000098  ...  0.000180   
2021-03-24      0.001133  0.000874  0.000354  0.000885  ...  0.000090   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-03-22      0.001015  0.000895  0.002442   0.002008  0.003334  0.001556   
2021-03-23      0.000508  0.000775  0.001948   0.001511  0.001950  0.001252   
2021-03-24      0.000254  0.000388  0.003151   0.001008  0.003099  0.001100   

Province_State                                     
Admin2         Unassigned      Washakie    Weston  
2021-03-22           -1.0  5.891664e-07  0.000003  
2021-03-23           -1.0  2.945832e-07  0.000794  
2021-03-24           -1.0  5.625770e-04  0.000397  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-03-22,0.000473,0.000358,NaN,0.000805,0.001094,0.001891,0.000147,0.000754,0.001759,NaN,...,0.000402,0.002036,0.001015,0.001346,3.789561e-14,0.000641,0.001069,0.002732,0.000513,0.000023
2021-03-23,0.000907,0.003405,NaN,0.001985,0.000817,0.002473,0.000403,0.000632,0.002186,NaN,...,0.001652,0.001871,0.004145,0.002956,1.894781e-14,0.000814,0.000824,0.001557,0.000257,0.001455
2021-03-24,0.002224,0.001702,NaN,0.002302,0.001580,0.002387,0.000449,0.000762,0.001093,NaN,...,0.002742,0.002640,0.003276,0.006024,9.473903e-15,0.000703,0.001762,0.001927,0.001717,0.000727


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                              \
Admin2           Autauga   Baldwin   Barbour          Bibb    Blount   
2021-03-22      0.000822  0.000161  0.000318 -1.268356e-07  0.000243   
2021-03-23      0.005513  0.001770  0.000159 -6.341779e-08  0.000122   
2021-03-24      0.002757  0.005935  0.000080 -3.170890e-08  0.003907   

Province_State                                                        ...  \
Admin2           Bullock        Butler   Calhoun  Chambers  Cherokee  ...   
2021-03-22      0.000007  9.752171e-10  0.000322  0.000344  0.000005  ...   
2021-03-23      0.000003  4.876086e-10  0.001811  0.000172  0.000002  ...   
2021-03-24      0.000002  2.438043e-10  0.000905  0.004472  0.000001  ...   

Province_State   Wyoming                                                      \
Admin2              Park        Platte  Sheridan      Sublette    Sweetwater   
2021-03-22      0.000005  1.399826e-18  0.000260  1.144267e-24  4.127247e-08   
2021-03-23      0.017244  6.999131e-19  0.000130  5.721333e-25  2.063623e-08   
2021-03-24      0.008622  3.499566e-19  0.000065  2.860666e-25  1.031812e-08   

Province_State                                                       \
Admin2                 Teton         Uinta Unassigned      Washakie   
2021-03-22      3.652714e-12  1.856287e-19  -0.501961  7.109521e-17   
2021-03-23      1.826357e-12  9.281435e-20  -0.501961  3.554760e-17   
2021-03-24      9.131785e-13  4.640717e-20  -0.501961  1.777380e-17   

Province_State                
Admin2                Weston  
2021-03-22      1.509903e-15  
2021-03-23      7.549517e-16  
2021-03-24      3.774758e-16  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
4,20210307,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
2021-03-07,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
2021-03-07,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN
